In [1]:
import matplotlib.pyplot as plt # for plotting maps
import maup # mggg's library for proration, see documentation here: https://github.com/mggg/maup
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
from statistics import mean, median
from functools import reduce

**Load Election Results and Turnout Data**

Importing the results from a file that connects the elections to 2016 VTDs

In [3]:
returns_2012 = pd.read_csv('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/ftp_election_data_16g/2012_General_Election_Returns.csv')
returns_2014 = pd.read_csv('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/ftp_election_data_16g/2014_General_Election_Returns.csv')
returns_2016 = pd.read_csv('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/ftp_election_data_16g/2016_General_Election_Returns.csv')

In [4]:
VRTO_2012 = pd.read_csv('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/ftp_election_data_16g/2012_General_Election_VRTO.csv')
VRTO_2014 = pd.read_csv('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/ftp_election_data_16g/2014_General_Election_VRTO.csv')
VRTO_2016 = pd.read_csv('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/ftp_election_data_16g/2016_General_Election_VRTO.csv')

**Load VTD Shapefile**

2016 VTD Shapefile is used for these results

In [128]:
vtd_2016 = gp.read_file('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/vtd16g/vtd16g.shp')

**Load Census Data**

2010 Census Data tied to 2016 VTDs

In [107]:
census_data = pd.read_excel('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/Pop2010_VTD_2016G.xlsx')

**Load TX-House, TX-Sen, and US-House info**

In [179]:
us_house = gp.read_file('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Original Files/house_sen/tl_2016_us_cd115/tl_2016_us_cd115.shp')
tx_sen = gp.read_file('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Original Files/house_sen/PLANS172/PLANS172.shp')
tx_house = gp.read_file('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Original Files/house_sen/PLANH407/PLANH407.shp')
print(tx_sen.count())
print(tx_house.count())

District    31
geometry    31
dtype: int64
District    150
geometry    150
dtype: int64


**Load PGP File**

In [112]:
tx_pgp_output = gp.read_file('/Users/peterhorton/Documents/Redistricting_Data_Hub/Coding/pgp-tx/Better Files/TX_vtds/TX_vtds.shp')
print(tx_pgp_output.count())
print(tx_pgp_output.head())

CNTYVTD     8941
VTD         8941
WHITE       8941
OTHER       8941
HISPANIC    8941
TOTPOP      8941
VAP         8941
BLACK       8941
BLKHISP     8941
WVAP        8941
HISPVAP     8941
BHVAP       8941
BVAP        8941
OTHVAP      8941
COUNTY      8941
FIPS        8941
PRES12R     8941
PRES12D     8941
SEN12R      8941
SEN12D      8941
TOTVR12     8941
TOTTO12     8941
SEN14R      8941
SEN14D      8941
GOV14R      8941
GOV14D      8941
TOTVR14     8941
TOTTO14     8941
PRES16D     8941
PRES16R     8941
TOTVR16     8941
TOTTO16     8941
USCD        8941
SEND        8941
HD          8941
AREA        8941
PERIM       8941
geometry    8941
dtype: int64
  CNTYVTD   VTD   WHITE  OTHER  HISPANIC  TOTPOP     VAP  BLACK  BLKHISP  \
0   10001  0001  2053.0   89.0     401.0  3131.0  2341.0  606.0    989.0   
1   10002  0002  3171.0   64.0     245.0  3744.0  2732.0  272.0    509.0   
2   10003  0003  1443.0   32.0     151.0  1766.0  1345.0  140.0    291.0   
3   10004  0004  1962.0   40.0      7

**Pivot Election Data**

In [116]:
office_list_2012 = ["U.S. Sen","President"]
party_list = ["R","D"]
returns_2012_filtered = returns_2012[returns_2012['Office'].isin(office_list_2012)]
returns_2012_filtered = returns_2012_filtered[returns_2012_filtered['Party'].isin(party_list)]
pivoted_2012 = pd.pivot_table(returns_2012_filtered, values=['Votes'], index=["cntyvtd"],columns=["Office","Party"])
pivoted_2012.columns = ["PRES12D","PRES12R","SEN12D","SEN12R"]
pivoted_2012.index.names = ["CNTYVTD"]
#print(pivoted_2012.head())
#print(pivoted_2012.count())

In [117]:
office_list_2014 = ["U.S. Sen","Governor"]
party_list = ["R","D"]
returns_2014_filtered = returns_2014[returns_2014['Office'].isin(office_list_2014)]
returns_2014_filtered = returns_2014_filtered[returns_2014_filtered['Party'].isin(party_list)]
pivoted_2014 = pd.pivot_table(returns_2014_filtered, values=['Votes'], index=["cntyvtd"],columns=["Office","Party"])
pivoted_2014.columns = ["GOV14D","GOV14R","SEN14D","SEN14R"]
pivoted_2014.index.names = ["CNTYVTD"]
#print(pivoted_2014.head())
#print(pivoted_2014.count())

In [118]:
office_list_2016 = ["President"]
party_list = ["R","D"]
returns_2016_filtered = returns_2016[returns_2016['Office'].isin(office_list_2016)]
returns_2016_filtered = returns_2016_filtered[returns_2016_filtered['Party'].isin(party_list)]
pivoted_2016 = pd.pivot_table(returns_2016_filtered, values=['Votes'], index=["cntyvtd"],columns=["Office","Party"])
pivoted_2016.columns = ["PRES16D","PRES16R"]
pivoted_2016.index.names = ["CNTYVTD"]
#print(pivoted_2016.head())
#print(pivoted_2016.count())

**Clean Turnout Data**

In [100]:
VRTO_2012_filtered = VRTO_2012[["CNTYVTD","TotalVR","TotalTO"]]
VRTO_2012_filtered = VRTO_2012_filtered.rename(columns={"TotalVR": "TOTVR12", "TotalTO": "TOTTO12"})
VRTO_2014_filtered = VRTO_2014[["CNTYVTD","TotalVR","TotalTO"]]
VRTO_2014_filtered = VRTO_2014_filtered.rename(columns={"TotalVR": "TOTVR14", "TotalTO": "TOTTO14"})
VRTO_2016_filtered = VRTO_2016[["CNTYVTD","TotalVR","TotalTO"]]
VRTO_2016_filtered = VRTO_2016_filtered.rename(columns={"TotalVR": "TOTVR16", "TotalTO": "TOTTO16"})

**Clean Census Data**

In [119]:
census_data = census_data[["VTD","CNTYVTD","e_ang","e_oth","e_hsp","e_total","e_vap","e_blak","e_bh","e_angvap","e_hspvap","e_bhvap","e_blakvap","e_othvap"]]

**Combine Turnout, Election, and Census Data**

In [131]:
all_voter_data = [census_data,pivoted_2012,VRTO_2012_filtered,pivoted_2014,VRTO_2014_filtered,pivoted_2016,VRTO_2016_filtered,vtd_2016]
merged_voter_data = reduce(lambda  left,right: pd.merge(left,right,on=['CNTYVTD'], how='outer'), all_voter_data)
print(merged_voter_data.count())

VTD_x         8941
CNTYVTD       8941
e_ang         8941
e_oth         8941
e_hsp         8941
e_total       8941
e_vap         8941
e_blak        8941
e_bh          8941
e_angvap      8941
e_hspvap      8941
e_bhvap       8941
e_blakvap     8941
e_othvap      8941
PRES12D       8941
PRES12R       8941
SEN12D        8941
SEN12R        8941
TOTVR12       8941
TOTTO12       8941
GOV14D        8941
GOV14R        8941
SEN14D        8941
SEN14R        8941
TOTVR14       8941
TOTTO14       8941
PRES16D       8941
PRES16R       8941
TOTVR16       8941
TOTTO16       8941
CNTY          8941
COLOR         8941
VTD_y         8941
VTDKEY        8941
Shape_area    8941
Shape_len     8941
geometry      8941
dtype: int64


**Assign to House Districts**

In [180]:
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
merged_voter_data = gp.GeoDataFrame(merged_voter_data, geometry='geometry')

proj = vtd_2016.crs
vtd_2016 = vtd_2016.to_crs(proj)

merged_voter_data.set_crs(proj)
merged_voter_data["geometry"] = merged_voter_data.buffer(0)

tx_sen.set_crs(proj)
tx_sen["geometry"] = tx_sen.buffer(0)

tx_house.set_crs(proj)
tx_house["geometry"] = tx_house.buffer(0)

us_house = us_house.to_crs(epsg=3081)
us_house["geometry"] = us_house.buffer(0)

us_house = us_house[us_house['STATEFP']=='48']
print(us_house.head())
print(us_house.crs)
print(us_house.count())

    STATEFP CD115FP GEOID                  NAMELSAD LSAD CDSESSN  MTFCC  \
44       48      01  4801  Congressional District 1   C2     115  G5200   
87       48      02  4802  Congressional District 2   C2     115  G5200   
125      48      03  4803  Congressional District 3   C2     115  G5200   
161      48      04  4804  Congressional District 4   C2     115  G5200   
191      48      05  4805  Congressional District 5   C2     115  G5200   

    FUNCSTAT        ALAND      AWATER     INTPTLAT      INTPTLON  \
44         N  20379602044   951776362  +31.9516781  -094.5520460   
87         N    799720185    54789657  +30.0473686  -095.1782923   
125        N   1245661528    97740060  +33.1594063  -096.6130717   
161        N  26237731127  1010312804  +33.2982880  -095.4180633   
191        N  13064499072   494023014  +32.2257951  -095.7316676   

                                              geometry  
44   POLYGON ((1412302.784 1176455.798, 1412346.279...  
87   POLYGON ((1419867.628

In [177]:
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
tx_sen_assign = maup.assign(merged_voter_data, tx_sen)
tx_house_assign = maup.assign(merged_voter_data, tx_house)


In [174]:
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
us_house_assign = maup.assign(merged_voter_data, us_house)

In [176]:
print(tx_sen_assign.head())
print(tx_house_assign.head())
print(us_house_assign.head())

merged_voter_data['USCD']=us_house_assign
merged_voter_data['SEND']=tx_house_assign
merged_voter_data['HD']=tx_house_assign

print(merged_voter_data.head())

0    23
1    17
2    30
3    20
4    24
dtype: int64
0     19
1     29
2     86
3     41
4    120
dtype: int64
0    391
1    399
2    328
3    401
4    375
dtype: int64
  VTD_x  CNTYVTD  e_ang  e_oth  e_hsp  e_total  e_vap  e_blak  e_bh  e_angvap  \
0  0008   530008   1117     19    131     1269    993       2   133       901   
1  0024  4690024   1357      8    276     1679   1260      38   314      1044   
2  0302  4210302    372     10    125      510    364       3   128       272   
3  0340  4790340    336     87   2839     3274   2460      17  2851       275   
4  4046   294046   2038    225   1012     3644   3083     416  1381      1839   

   ...  CNTY  COLOR  VTD_y  VTDKEY    Shape_area      Shape_len  \
0  ...    53      2   0008       0  6.717692e+07   55772.006345   
1  ...   469      2   0024       0  5.426296e+07   50831.107001   
2  ...   421      6   0302       0  4.890835e+08  115435.448089   
3  ...   479      1   0340       0  1.949293e+06    5991.528062   
4  ...   